# Regression Model to Predict Average Price per Kilogram

This model's task is to predict the average price per kilogram and stock on hand for **Golden Delicious Apples**.

<img src="https://cdn.shopify.com/s/files/1/0271/7823/8041/products/golden-delicious-apple_2048x2048.jpg" alt="Golden Delicious Apples" title="Golden Delicious Apples" />

Providing an accurate and robust solution has immense economic potential, for users to be able to accurately determine how much stock to have on hand and what price to set the Golden Delicious Apples to, using a set of features (Weight, Low Price, High Price, etc.).


## Imports

Importing:
- numpy for mathemticial calculations
- pandas to read and store data as Data Frames
- seasborn and matplotlib for visualisations
- scikit learn for Regression Models, Model Selection, Pre-processing and Metrics
- scipy for Stats
- pickle to save the model

In [1]:
# Numpy and Pandas
import numpy as np
import pandas as pd

# Scikit learn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Visualisations
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#Extra Modules
from scipy import stats
import pickle

## Importing Data

In [12]:
train = pd.read_csv('df-train_set.csv')
test = pd.read_csv('df-test_set.csv')

## Exploring Data

Firstly, we take a quick look at the data from our training set

In [6]:
train.head()

,Province,Container,Size_Grade,Weight_Kg,Commodities,Date,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,avg_price_per_kg
0,CAPE,EC120,1L,12.0,APPLE GRANNY SMITH,2020-03-10,108.0,112.0,3236.0,29,348.0,0,9.30
1,CAPE,M4183,1L,18.3,APPLE GOLDEN DELICIOUS,2020-09-09,150.0,170.0,51710.0,332,6075.6,822,8.51
2,GAUTENG,AT200,1L,20.0,AVOCADO PINKERTON,2020-05-05,70.0,80.0,4860.0,66,1320.0,50,3.68
3,TRANSVAAL,BJ090,1L,9.0,TOMATOES-LONG LIFE,2020-01-20,60.0,60.0,600.0,10,90.0,0,6.67
4,WESTERN FREESTATE,PP100,1R,10.0,POTATO SIFRA (WASHED),2020-07-14,40.0,45.0,41530.0,927,9270.0,393,4.48


In [13]:
train.shape

(64376, 13)

The data frame has 64376 rows and 13 columns

As shown above, it is easy to see what data type each column has but the **info** method can confirm that.

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64376 entries, 0 to 64375
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Province          64376 non-null  object 
 1   Container         64376 non-null  object 
 2   Size_Grade        64376 non-null  object 
 3   Weight_Kg         64376 non-null  float64
 4   Commodities       64376 non-null  object 
 5   Date              64376 non-null  object 
 6   Low_Price         64376 non-null  float64
 7   High_Price        64376 non-null  float64
 8   Sales_Total       64376 non-null  float64
 9   Total_Qty_Sold    64376 non-null  int64  
 10  Total_Kg_Sold     64376 non-null  float64
 11  Stock_On_Hand     64376 non-null  int64  
 12  avg_price_per_kg  64376 non-null  float64
dtypes: float64(6), int64(2), object(5)
memory usage: 6.4+ MB


The display of each column above was enough to show that none of the columns contains null or missing values. 

### Removing unneccessary features
Just by looking at the Data Frame, it seems that there are features that are not required. 
To start with, the model's focus is on the **Golden Delicious Apples**, therefore the other commodities could be filtered out.
Secondly, the **Province** and **Container** are not required because these two features have nothing to do with the average price <br>
<br><br>
After filtering, we know that all rows are **Golden Delicious Apples** Commodities, therefore the **Commodities** feature is also not required.

In [14]:
train = train.query('Commodities == "APPLE GOLDEN DELICIOUS"')
train.drop(['Province', 'Container', 'Commodities'], inplace = True, axis = 1)
train.head()

,Size_Grade,Weight_Kg,Date,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,avg_price_per_kg
1,1L,18.3,2020-09-09,150.0,170.0,51710.0,332,6075.6,822,8.51
7,2M,11.0,2020-04-14,50.0,50.0,16000.0,320,3520.0,0,4.55
24,2S,9.0,2020-04-16,55.0,55.0,990.0,18,162.0,1506,6.11
40,1S,18.3,2020-05-04,80.0,120.0,32020.0,388,7100.4,443,4.51
69,1S,400.0,2020-09-28,1800.0,1800.0,1800.0,1,400.0,2,4.50


In [15]:
train_y = train.pop('avg_price_per_kg')

In [17]:
train.head()

,Size_Grade,Weight_Kg,Date,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand
1,1L,18.3,2020-09-09,150.0,170.0,51710.0,332,6075.6,822
7,2M,11.0,2020-04-14,50.0,50.0,16000.0,320,3520.0,0
24,2S,9.0,2020-04-16,55.0,55.0,990.0,18,162.0,1506
40,1S,18.3,2020-05-04,80.0,120.0,32020.0,388,7100.4,443
69,1S,400.0,2020-09-28,1800.0,1800.0,1800.0,1,400.0,2
